In [6]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fe

In [7]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-04 19:00:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.25MB/s    in 0.7s    

2023-03-04 19:01:00 (1.25 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [8]:
# Import Spark and create a SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=7cf3eb8362b776036157dda7ea24814b0efe98456e1a64899aa5f3f6c9ab4c66
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


# Extract the Amazon Data into Spark DataFrame

In [9]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles


In [18]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

In [19]:
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

# Transform the Data

## Create the "review_id_table".

In [27]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = df.withColumn("review_date", to_date(df["review_date"])).select(['review_id','customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R14LVB34Z2Z53I|   10293599|B000YMOQZY|     262935067| 2015-08-31|
|R2M0R9CWPC82LT|    6087195|B00ISAEC80|     452318038| 2015-08-31|
| RQNQI2R3BM52A|   33717063|B00VBB6AVU|     675969129| 2015-08-31|
| RTUIX1135HUAB|   14948020|B008A5POJO|     282925893| 2015-08-31|
| RE3QSAT8TL010|   21694522|B014CW2RN4|     109077793| 2015-08-31|
|R14TN65IY0NUOD|   14948020|B00DRA3EOY|     959245080| 2015-08-31|
|R3LIF8MCNH1ODF|   20031966|B009G3U0PW|     762436916| 2015-08-31|
|R33X0DGA4PPQ3L|   12223745|B013XBYQOS|     137980344| 2015-08-31|
|R2DJJP7DQRT1ZW|   30231876|B003L5H25W|     165995375| 2015-08-31|
|R2A8XDXW5XOT4H|   51714365|B00136NVH4|     227508666| 2015-08-31|
|R3MJK3L7ER61TT|   37216001|B0030AKPQ6|     164096943| 2015-08-31|
|R38Z7XMCVDR0R5|   49207156|B014EQE0V0|     714368268| 2015-08

In [33]:
products_df = df.select(['product_id','product_title']).dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B014PO1708|       See You Again|
|B001APIS9Y|What Happened to ...|
|B005XT4CAW|I Can Dream About...|
|B006WH9FPQ|  If You Want To Fly|
|B0026G2MG6|Ghetto Fabolous [...|
|B0096RUQZW|       Call Me Maybe|
|B003YZGVFQ|Maxwell's silver ...|
|B00138B7N2|He Touched Me (Al...|
|B00OLF0U3M|Beyond The Lights...|
|B007D89P76|A Hampshire Suite...|
|B0011U5E1S|             Nirvana|
|B00JWFMVNS|Sousa (Red Classics)|
|B00120EDBY|Talkin' To You Ey...|
|B00IWI3HNO|          Sweet Life|
|B00YIB4KJY|                Fall|
|B0011ZVFIO|Carmel City (LP V...|
|B00C32RB4Q|         Lola Montez|
|B001NTZ4NA|Thug Lovin' (Albu...|
|B001BHJZN4|      Eye In The Sky|
|B005SR8QO2|      My Kinda Party|
+----------+--------------------+
only showing top 20 rows



In [30]:
customers_df = df.select(['customer_id', 'product_id', 'product_parent', 'review_date']).groupBy('customer_id').count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   39290568|    1|
|   23601866|    1|
|   19004376|    1|
|   16320823|    1|
|   13973119|    1|
|    1155154|    1|
|   24317024|    2|
|   15125275|    1|
|   35781732|    2|
|   40985731|   12|
|   19522542|    1|
|   50243180|    1|
|    2702985|    1|
|   41857916|    1|
|    1943933|    2|
|   42050930|    5|
|    4867895|    1|
|   46317087|    3|
|   52021773|    2|
|   15504622|    9|
+-----------+-----+
only showing top 20 rows



In [31]:
vine_df = df.select(['review_id','star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R14LVB34Z2Z53I|          5|            0|          0|   N|
|R2M0R9CWPC82LT|          5|            0|          0|   N|
| RQNQI2R3BM52A|          4|            0|          0|   N|
| RTUIX1135HUAB|          5|            0|          0|   N|
| RE3QSAT8TL010|          4|            0|          0|   N|
|R14TN65IY0NUOD|          5|            0|          0|   N|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|
|R3MJK3L7ER61TT|          5|            0|          0|   N|
|R38Z7XMCVDR0R5|          5|            1|          1|   N|
|R3FFESBQXPR7S6|          4|            0|          0|   N|
|R3QO6Z942CKH34|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_df', mode=mode, properties=config)
products_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)
customers_df.write.jdbc(url=jdbc_url, table='customers_df', mode=mode, properties=config)
vine_df.write.jdbc(url=jdbc_url, table='vine_df', mode=mode, properties=config)
